In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-ibm-transpiler

*推定QPU使用量: なし（注意: このチュートリアルはトランスパイレーションに焦点を当てているため、ジョブの実行は行いません）*

## 背景
**Qiskit AI搭載トランスパイラサービス（QTS）** は、ルーティングパスと合成パスの両方に機械学習ベースの最適化を導入しています。これらのAIモードは、従来のトランスパイレーションの限界、特に大規模回路や複雑なハードウェアトポロジーに対する課題を克服するために設計されています。

**2025年7月**時点で、**トランスパイラサービス**は新しいIBM Quantum&reg; Platformに移行されており、以前の形式では利用できなくなっています。トランスパイラサービスの最新の状況については、[トランスパイラサービスのドキュメント](/guides/qiskit-transpiler-service)をご参照ください。AIトランスパイラは、標準的なQiskitトランスパイレーションと同様に、ローカルで引き続き使用できます。`generate_preset_pass_manager()` を `generate_ai_pass_manager()` に置き換えるだけです。この関数は、AI搭載のルーティングパスと合成パスをローカルのトランスパイレーションワークフローに直接統合するパスマネージャーを構築します。

### AIパスの主な機能
- ルーティングパス: AI搭載のルーティングは、特定の回路とバックエンドに基づいて量子ビットのパスを動的に調整し、過剰なSWAPゲートの必要性を低減します。
    - `AIRouting`: レイアウト選択と回路ルーティング

- 合成パス: AI技術は多量子ビットゲートの分解を最適化し、通常エラーが発生しやすい2量子ビットゲートの数を最小限に抑えます。
    - `AICliffordSynthesis`: Cliffordゲートの合成
    - `AILinearFunctionSynthesis`: 線形関数回路の合成
    - `AIPermutationSynthesis`: 順列回路の合成
    - `AIPauliNetworkSynthesis`: Pauliネットワーク回路の合成（Qiskitトランスパイラサービスでのみ利用可能で、ローカル環境では利用できません）

- 従来のトランスパイレーションとの比較: 標準的なQiskitトランスパイラは、幅広い量子回路を効果的に処理できる堅牢なツールです。しかし、回路の規模が大きくなったり、ハードウェア構成が複雑になったりすると、AIパスは追加の最適化効果を発揮できます。ルーティングと合成に学習済みモデルを使用することで、QTSは回路レイアウトをさらに洗練させ、困難な大規模量子タスクのオーバーヘッドを削減します。

このチュートリアルでは、ルーティングパスと合成パスの両方を使用してAIモードを評価し、AIがパフォーマンス向上をもたらす箇所を明らかにするために、従来のトランスパイレーションとの結果を比較します。

利用可能なAIパスの詳細については、[AIパスのドキュメント](/guides/ai-transpiler-passes)をご覧ください。

### なぜ量子回路のトランスパイレーションにAIを使用するのか？
量子回路の規模と複雑さが増すにつれて、従来のトランスパイレーション手法では、レイアウトの最適化やゲート数の削減を効率的に行うことが困難になります。特に数百量子ビットを含む大規模な回路は、デバイスの制約、限られた接続性、量子ビットのエラー率により、ルーティングと合成に大きな課題をもたらします。

ここで、AI搭載のトランスパイレーションが有力な解決策となります。機械学習技術を活用することで、QiskitのAI搭載トランスパイラは量子ビットのルーティングとゲート合成についてより賢明な判断を行い、大規模量子回路のより良い最適化を実現します。

### ベンチマーク結果の概要
![Graph showing AI transpiler performance against Qiskit](../docs/images/tutorials/ai-transpiler-introduction/ai-transpiler-benchmarks.avif)

ベンチマークテストでは、AIトランスパイラは標準的なQiskitトランスパイラと比較して、一貫してより浅く高品質な回路を生成しました。これらのテストでは、[`generate_preset_passmanager`]で構成されたQiskitのデフォルトパスマネージャー戦略を使用しました。このデフォルト戦略は多くの場合効果的ですが、より大規模で複雑な回路では困難が生じることがあります。一方、AI搭載パスは、IBM Quantumハードウェアのheavy-hexトポロジーにトランスパイルする場合、大規模回路（100量子ビット以上）において、2量子ビットゲート数の平均24%削減と回路深度の平均36%削減を達成しました。これらのベンチマークの詳細については、こちらの[ブログ](https://www.ibm.com/quantum/blog/qiskit-performance)をご参照ください。

このチュートリアルでは、AIパスの主な利点と従来の手法との比較について解説します。

In [1]:
# This cell is hidden from users;
# it just disables a linting rule.
# ruff: noqa: F811

## 前提条件

このチュートリアルを始める前に、以下がインストールされていることを確認してください：

* Qiskit SDK v1.0以降、[visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22以降
* Qiskit IBM&reg; Transpiler AIローカルモード付き (`pip install 'qiskit-ibm-transpiler[ai-local-mode]'`)

## セットアップ

In [2]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import efficient_su2, PermutationGate
from qiskit.synthesis.qft import synth_qft_full
from qiskit.circuit.random import random_circuit, random_clifford_circuit
from qiskit.transpiler import generate_preset_pass_manager, CouplingMap
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_transpiler import generate_ai_pass_manager
from qiskit.synthesis.permutation import (
    synth_permutation_depth_lnn_kms,
    synth_permutation_basic,
)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import logging

seed = 42


# Used for generating permutation circuits in part two for comparison
def generate_permutation_circuit(width, pattern):
    circuit = QuantumCircuit(width)
    circuit.append(
        PermutationGate(pattern=pattern),
        qargs=range(width),
    )
    return circuit


# Creates a Bernstein-Vazirani circuit given the number of qubits
def create_bv_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits, num_qubits - 1)
    qc.x(num_qubits - 1)
    qc.h(qc.qubits)
    for i in range(num_qubits - 1):
        qc.cx(i, num_qubits - 1)
    qc.h(qc.qubits[:-1])
    return qc


# Transpile a circuit with a given pass manager and return metrics
def transpile_with_metrics(pass_manager, circuit):
    start = time.time()
    qc_out = pass_manager.run(circuit)
    elapsed = time.time() - start

    depth_2q = qc_out.depth(lambda x: x.operation.num_qubits == 2)
    gate_count = qc_out.size()

    return qc_out, {
        "depth_2q": depth_2q,
        "gate_count": gate_count,
        "time_s": elapsed,
    }


# Used for collecting metrics for part 3 of synthesis methods
def synth_transpile_with_metrics(qc, pm, pattern_id, method):
    start = time.time()
    qc = pm.run(qc)
    elapsed = time.time() - start

    return {
        "Pattern": pattern_id,
        "Method": method,
        "Depth (2Q)": qc.depth(lambda x: x.operation.num_qubits == 2),
        "Gates": qc.size(),
        "Time (s)": elapsed,
    }


# Ignore logs like "INFO:qiskit_ibm_transpiler.wrappers.ai_local_synthesis:Running Linear Functions AI synthesis on local mode"

logging.getLogger(
    "qiskit_ibm_transpiler.wrappers.ai_local_synthesis"
).setLevel(logging.WARNING)

# パート I. Qiskitパターン

それでは、Qiskitパターンを使用して、シンプルな量子回路でAIトランスパイラサービスを使用する方法を見てみましょう。重要なポイントは、標準の `generate_preset_pass_manager()` の代わりに `generate_ai_pass_manager()` で `PassManager` を作成することです。

## ステップ 1: 古典的な入力を量子問題にマッピングする

このセクションでは、広く使用されているハードウェア効率の良いアンザッツである `efficient_su2` 回路でAIトランスパイラをテストします。この回路は、変分量子アルゴリズム（例：VQE）や量子機械学習タスクに特に関連しており、トランスパイレーション性能を評価するための理想的なテストケースとなります。

`efficient_su2` 回路は、単一量子ビット回転とCNOTなどのエンタングルメントゲートの交互の層で構成されています。これらの層により、ゲート深度を管理可能に保ちながら、量子状態空間の柔軟な探索が可能になります。この回路を最適化することで、ゲート数の削減、フィデリティの向上、ノイズの最小化を目指します。これにより、AIトランスパイラの効率をテストするための優れた候補となります。

In [3]:
# For our transpilation, we will use a large circuit of 101 qubits
qc = efficient_su2(90, entanglement="circular", reps=1).decompose()

# Draw a smaller version of the circuit to get a visual representation
qc_small = efficient_su2(5, entanglement="circular", reps=1).decompose()
qc_small.draw(output="mpl")

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/c6e9c2c0-e02c-4276-bae8-d5692e60b6b8-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/c6e9c2c0-e02c-4276-bae8-d5692e60b6b8-0.avif)

## ステップ 2: 量子ハードウェア実行のための問題の最適化
### バックエンドの選択
この例では、シミュレータではなく、少なくとも100量子ビットを持つ、最も空いている稼働中のIBM Quantumバックエンドを選択します：

**注意:** 最も空いているバックエンドは時間とともに変わる可能性があるため、実行ごとに異なるデバイスが選択される場合があります。カップリングマップなどのデバイス固有のプロパティにより、トランスパイルされた回路に違いが生じることがあります。

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=100
)
cm = backend.coupling_map
print(f"Using backend: {backend.name}")

Using backend: ibm_torino


### Create AI and traditional pass managers
To evaluate the effectiveness of the AI transpiler, we will perform two transpilation runs. First, we will transpile the circuit using the AI transpiler. Then, we will run a comparison by transpiling the same circuit without the AI transpiler, using traditional methods. Both transpilation processes will use the same coupling map from the chosen backend and the optimization level set to 3 for a fair comparison.

Both of these methods reflect the standard approach to create `PassManager` instances to transpile circuits in Qiskit.

In [5]:
pm_ai = generate_ai_pass_manager(
    optimization_level=3,
    ai_optimization_level=3,
    coupling_map=cm,
    include_ai_synthesis=True,  # used for part 3 when comparing synthesis methods
)

pm_no_ai = generate_preset_pass_manager(
    optimization_level=3,
    coupling_map=cm,
    seed_transpiler=seed,  # note that the AI pass manager does not currently support seeding
)

### AIパスマネージャーと従来のパスマネージャーの作成
AIトランスパイラの有効性を評価するために、2回のトランスパイレーションを実行します。まず、AIトランスパイラを使用して回路をトランスパイルします。次に、AIトランスパイラを使用せず、従来の手法で同じ回路をトランスパイルして比較を行います。公正な比較のため、両方のトランスパイレーションプロセスは、選択したバックエンドの同じカップリングマップを使用し、最適化レベルを3に設定します。

これらの方法はどちらも、Qiskitで回路をトランスパイルするための `PassManager` インスタンスを作成する標準的なアプローチを反映しています。

In [6]:
# Transpile using standard (non-AI) pass manager
_, metrics_no_ai = transpile_with_metrics(pm_no_ai, qc)
print(
    f"Standard transpilation: Depth (2q) {metrics_no_ai['depth_2q']}, "
    f"Gate count {metrics_no_ai['gate_count']}, Time {metrics_no_ai['time_s']}"
)

# Transpile using AI pass manager
_, metrics_ai = transpile_with_metrics(pm_ai, qc)
print(
    f"AI transpilation      : Depth (2q) {metrics_ai['depth_2q']}, "
    f"Gate count {metrics_ai['gate_count']}, Time {metrics_ai['time_s']}"
)

Standard transpilation: Depth (2q) 95, Gate count 458, Time 0.04650712013244629
AI transpilation      : Depth (2q) 90, Gate count 456, Time 0.9342479705810547


回路をトランスパイルし、時間を記録します。

In [7]:
# Circuits to benchmark
seed = 42
circuits = [
    {
        "name": "Random",
        "qc": random_circuit(num_qubits=30, depth=10, seed=seed),
    },
    {
        "name": "Clifford",
        "qc": random_clifford_circuit(
            num_qubits=40, num_gates=200, seed=seed
        ),
    },
    {
        "name": "QFT",
        "qc": synth_qft_full(num_qubits=20, do_swaps=False).decompose(),
    },
    {
        "name": "BV",
        "qc": create_bv_circuit(40),
    },
]

results = []

# Run the transpilation for each circuit and store the results
for circuit in circuits:
    qc_no_ai, metrics_no_ai = transpile_with_metrics(pm_no_ai, circuit["qc"])
    qc_ai, metrics_ai = transpile_with_metrics(pm_ai, circuit["qc"])

    print("Completed transpilation for", circuit["name"])

    results.append(
        {
            "Circuit": circuit["name"],
            "Depth 2Q (No AI)": metrics_no_ai["depth_2q"],
            "Gate Count (No AI)": metrics_no_ai["gate_count"],
            "Time (No AI)": metrics_no_ai["time_s"],
            "Depth 2Q (AI)": metrics_ai["depth_2q"],
            "Gate Count (AI)": metrics_ai["gate_count"],
            "Time (AI)": metrics_ai["time_s"],
        }
    )

df = pd.DataFrame(results)
df

Completed transpilation for Random
Completed transpilation for Clifford
Completed transpilation for QFT
Completed transpilation for BV


,Circuit,Depth 2Q (No AI),Gate Count (No AI),Time (No AI),Depth 2Q (AI),Gate Count (AI),Time (AI)
0,Random,37,221,0.039347,24,181,0.773718
1,Clifford,36,232,0.036633,43,267,1.097431
2,QFT,165,924,0.077458,130,913,3.660771
3,BV,65,155,0.024993,70,155,0.345522


Average percentage reduction for each metric. Positive are improvements, negative are degradations.

In [8]:
# Average reduction from non-AI to AI transpilation as a percentage
avg_reduction_depth = (
    (df["Depth 2Q (No AI)"] - df["Depth 2Q (AI)"]).mean()
    / df["Depth 2Q (No AI)"].mean()
    * 100
)
avg_reduction_gates = (
    (df["Gate Count (No AI)"] - df["Gate Count (AI)"]).mean()
    / df["Gate Count (No AI)"].mean()
    * 100
)
avg_reduction_time = (
    (df["Time (No AI)"] - df["Time (AI)"]).mean()
    / df["Time (No AI)"].mean()
    * 100
)

print(f"Average reduction in depth: {avg_reduction_depth:.2f}%")
print(f"Average reduction in gate count: {avg_reduction_gates:.2f}%")
print(f"Average reduction in transpilation time: {avg_reduction_time:.2f}%")

Average reduction in depth: 11.88%
Average reduction in gate count: 1.04%
Average reduction in transpilation time: -3193.95%


In [9]:
fig, axs = plt.subplots(1, 3, figsize=(21, 6))
df.plot(
    x="Circuit",
    y=["Depth 2Q (No AI)", "Depth 2Q (AI)"],
    kind="bar",
    ax=axs[0],
)
axs[0].set_title("Circuit Depth Comparison")
axs[0].set_ylabel("Depth")
axs[0].set_xlabel("Circuit")
axs[0].tick_params(axis="x", rotation=45)
df.plot(
    x="Circuit",
    y=["Gate Count (No AI)", "Gate Count (AI)"],
    kind="bar",
    ax=axs[1],
)
axs[1].set_title("Gate Count Comparison")
axs[1].set_ylabel("Gate Count")
axs[1].set_xlabel("Circuit")
axs[1].tick_params(axis="x", rotation=45)
df.plot(x="Circuit", y=["Time (No AI)", "Time (AI)"], kind="bar", ax=axs[2])
axs[2].set_title("Time Comparison")
axs[2].set_ylabel("Time (seconds)")
axs[2].set_xlabel("Circuit")
axs[2].tick_params(axis="x", rotation=45)
fig.suptitle(
    "Benchmarking AI transpilation vs Non-AI transpilation for various circuits"
)

plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/79b8d5d9-0f9d-42ca-9583-8bec17430014-0.avif" alt="Output of the previous code cell" />

The AI transpiler's performance varies significantly based on the type of circuit being optimized. In some cases, it achieves notable reductions in circuit depth and gate count compared to the standard transpiler. However, these improvements often come with a substantial increase in runtime.

For certain types of circuits, the AI transpiler may yield slightly better results in terms of circuit depth but may also lead to an increase in gate count and a significant runtime penalty. These observations suggest that the AI transpiler's benefits are not uniform across all circuit types. Instead, its effectiveness depends on the specific characteristics of the circuit, making it more suitable for some use cases than others.

## When should users choose AI-powered transpilation?

The AI-powered transpiler in Qiskit excels in scenarios where traditional transpilation methods struggle, particularly with large-scale and complex quantum circuits. For circuits involving hundreds of qubits or those targeting hardware with intricate coupling maps, the AI transpiler offers superior optimization in terms of circuit depth, gate count, and runtime efficiency. In benchmarking tests, it has consistently outperformed traditional methods, delivering significantly shallower circuits and reducing gate counts, which are critical for enhancing performance and mitigating noise on real quantum hardware.

Users should consider AI-powered transpilation when working with:
- Large circuits where traditional methods fail to efficiently handle the scale.
- Complex hardware topologies where device connectivity and routing challenges arise.
- Performance-sensitive applications where reducing circuit depth and improving fidelity are paramount.

# Part III. Explore AI-powered permutation network synthesis

Permutation networks are foundational in quantum computing, particularly for systems constrained by restricted topologies. These networks facilitate long-range interactions by dynamically swapping qubits to mimic all-to-all connectivity on hardware with limited connectivity. Such transformations are essential for implementing complex quantum algorithms on near-term devices, where interactions often span beyond nearest neighbors.

In this section, we highlight the synthesis of permutation networks as a compelling use case for the AI-powered transpiler in Qiskit. Specifically, the `AIPermutationSynthesis` pass leverages AI-driven optimization to generate efficient circuits for qubit permutation tasks. By contrast, generic synthesis approaches often struggle to balance gate count and circuit depth, especially in scenarios with dense qubit interactions or when attempting to achieve full connectivity.

We will walk through a Qiskit patterns example showcasing the synthesis of a permutation network to achieve all-to-all connectivity for a set of qubits. We will compare the performance of `AIPermutationSynthesis` against the standard synthesis methods in Qiskit. This example will demonstrate how the AI transpiler optimizes for lower circuit depth and gate count, highlighting its advantages in practical quantum workflows. To activate the AI synthesis pass, we will use the `generate_ai_pass_manager()` function with the `include_ai_synthesis` parameter set to `True`.

## Step 1: Map classical inputs to a quantum problem

To represent a classical permutation problem on a quantum computer, we start by defining the structure of the quantum circuits. For this example:

1. Quantum circuit initialization:
   We allocate 27 qubits to match the backend we will use, which has 27 qubits.

2. Apply permutations:
   We generate ten random permutation patterns (`pattern_1` through `pattern_10`) using a fixed seed for reproducibility. Each permutation pattern is applied to a separate quantum circuit (`qc_1` through `qc_10`).

3. Circuit decomposition:
   Each permutation operation is decomposed into native gate sets compatible with the target quantum hardware. We analyze the depth and the number of two-qubit gates (nonlocal gates) for each decomposed circuit.

The results provide insight into the complexity of representing classical permutation problems on a quantum device, demonstrating the resource requirements for different permutation patterns.

In [10]:
# Parameters
width = 27
num_circuits = 10

# Set random seed
np.random.seed(seed)


# Generate random patterns and circuits
patterns = [
    np.random.permutation(width).tolist() for _ in range(num_circuits)
]
circuits = {
    f"qc_{i}": generate_permutation_circuit(width, pattern)
    for i, pattern in enumerate(patterns, start=1)
}

# Display one of the circuits
circuits["qc_1"].decompose(reps=3).draw(output="mpl", fold=-1)

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/76a3e847-0808-4413-bd0c-c760cd2df3f4-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution
In this step, we proceed with optimization using the AI synthesis passes.

For the AI synthesis passes, the `PassManager` requires only the coupling map of the backend. However, it is important to note that not all coupling maps are compatible; only those that the `AIPermutationSynthesis` pass has been trained on will work. Currently, the `AIPermutationSynthesis` pass supports blocks of sizes 65, 33, and 27 qubits. For this example we use a 27-qubit QPU.

For comparison, we will evaluate the performance of AI synthesis against generic permutation synthesis methods in Qiskit, including:

- `synth_permutation_depth_lnn_kms`: This method synthesizes a permutation circuit for a linear nearest-neighbor (LNN) architecture using the Kutin, Moulton, and Smithline (KMS) algorithm. It guarantees a circuit with a depth of at most $ n $ and a size of at most $ n(n-1)/2 $, where both depth and size are measured in terms of SWAP gates.

- `synth_permutation_basic`: This is a straightforward implementation that synthesizes permutation circuits without imposing constraints on connectivity or optimization for specific architectures. It serves as a baseline for comparing performance with more advanced methods.

Each of these methods represents a distinct approach to synthesizing permutation networks, providing a comprehensive benchmark against the AI-powered methods.

For more details about synthesis methods in Qiskit, refer to the [Qiskit API documentation](/docs/api/qiskit/synthesis).

Define the coupling map representing the 27-qubit QPU.

In [11]:
coupling_map = [
    [1, 0],
    [2, 1],
    [3, 2],
    [3, 5],
    [4, 1],
    [6, 7],
    [7, 4],
    [7, 10],
    [8, 5],
    [8, 9],
    [8, 11],
    [11, 14],
    [12, 10],
    [12, 13],
    [12, 15],
    [13, 14],
    [16, 14],
    [17, 18],
    [18, 15],
    [18, 21],
    [19, 16],
    [19, 22],
    [20, 19],
    [21, 23],
    [23, 24],
    [25, 22],
    [25, 24],
    [26, 25],
]
CouplingMap(coupling_map).draw()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/84dff2c2-a496-4828-bb8e-08d373816a36-0.avif" alt="Output of the previous code cell" />

各メトリクスの平均削減率です。正の値は改善を、負の値は悪化を示します。

In [12]:
results = []
pm_no_ai_synth = generate_preset_pass_manager(
    coupling_map=cm,
    optimization_level=1,  # set to 1 since we are using the synthesis methods
)

# Transpile and analyze all circuits
for i, (qc_name, qc) in enumerate(circuits.items(), start=1):
    pattern = patterns[i - 1]  # Get the corresponding pattern

    qc_depth_lnn_kms = synth_permutation_depth_lnn_kms(pattern)
    qc_basic = synth_permutation_basic(pattern)

    # AI synthesis
    results.append(
        synth_transpile_with_metrics(
            qc.decompose(reps=3),
            pm_ai,
            qc_name,
            "AI",
        )
    )

    # Depth-LNN-KMS Method
    results.append(
        synth_transpile_with_metrics(
            qc_depth_lnn_kms.decompose(reps=3),
            pm_no_ai_synth,
            qc_name,
            "Depth-LNN-KMS",
        )
    )

    # Basic Method
    results.append(
        synth_transpile_with_metrics(
            qc_basic.decompose(reps=3),
            pm_no_ai_synth,
            qc_name,
            "Basic",
        )
    )


results_df = pd.DataFrame(results)

Record the metrics (depth, gate count, time) for each circuit after transpilation.

In [13]:
# Calculate averages for each metric
average_metrics = results_df.groupby("Method")[
    ["Depth (2Q)", "Gates", "Time (s)"]
].mean()
average_metrics = average_metrics.round(3)  # Round to two decimal places
print("\n=== Average Metrics ===")
print(average_metrics)

# Identify the best non-AI method based on least average depth
non_ai_methods = [
    method for method in results_df["Method"].unique() if method != "AI"
]
best_non_ai_method = average_metrics.loc[non_ai_methods][
    "Depth (2Q)"
].idxmin()
print(
    f"\nBest Non-AI Method (based on least average depth): {best_non_ai_method}"
)

# Compare AI to the best non-AI method
ai_metrics = average_metrics.loc["AI"]
best_non_ai_metrics = average_metrics.loc[best_non_ai_method]

comparison = {
    "Metric": ["Depth (2Q)", "Gates", "Time (s)"],
    "AI": [
        ai_metrics["Depth (2Q)"],
        ai_metrics["Gates"],
        ai_metrics["Time (s)"],
    ],
    best_non_ai_method: [
        best_non_ai_metrics["Depth (2Q)"],
        best_non_ai_metrics["Gates"],
        best_non_ai_metrics["Time (s)"],
    ],
    "Improvement (AI vs Best Non-AI)": [
        ai_metrics["Depth (2Q)"] - best_non_ai_metrics["Depth (2Q)"],
        ai_metrics["Gates"] - best_non_ai_metrics["Gates"],
        ai_metrics["Time (s)"] - best_non_ai_metrics["Time (s)"],
    ],
}

comparison_df = pd.DataFrame(comparison)
print("\n=== Comparison of AI vs Best Non-AI Method ===")
comparison_df


=== Average Metrics ===
               Depth (2Q)  Gates  Time (s)
Method                                    
AI                   23.9   82.8     0.248
Basic                29.8   91.0     0.012
Depth-LNN-KMS        70.8  531.6     0.017

Best Non-AI Method (based on least average depth): Basic

=== Comparison of AI vs Best Non-AI Method ===


,Metric,AI,Basic,Improvement (AI vs Best Non-AI)
0,Depth (2Q),23.900,29.800,-5.900
1,Gates,82.800,91.000,-8.200
2,Time (s),0.248,0.012,0.236


The results demonstrate that the AI transpiler outperforms all other Qiskit synthesis methods for this set of random permutation circuits. Key findings include:

1. Depth: The AI transpiler achieves the lowest average depth, indicating superior optimization of circuit layouts.
2. Gate count: It significantly reduces the number of gates compared to other methods, improving execution fidelity and efficiency.
3. Transpilation time: All methods run very quickly at this scale, making them practical for use. However, the AI transpiler does has a notable runtime increase compared to traditional methods due to the complexity of the AI models used.

These results establish the AI transpiler as the most effective approach for this benchmark, particularly for depth and gate count optimization.

Plot the results to compare the performance of the AI synthesis passes against the generic synthesis methods.

In [14]:
methods = results_df["Method"].unique()

fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Pivot the DataFrame and reorder columns to ensure AI is first
pivot_depth = results_df.pivot(
    index="Pattern", columns="Method", values="Depth (2Q)"
)[["AI", "Depth-LNN-KMS", "Basic"]]
pivot_gates = results_df.pivot(
    index="Pattern", columns="Method", values="Gates"
)[["AI", "Depth-LNN-KMS", "Basic"]]
pivot_time = results_df.pivot(
    index="Pattern", columns="Method", values="Time (s)"
)[["AI", "Depth-LNN-KMS", "Basic"]]

pivot_depth.plot(kind="bar", ax=axs[0], legend=False)
axs[0].set_title("Circuit Depth Comparison")
axs[0].set_ylabel("Depth")
axs[0].set_xlabel("Pattern")
axs[0].tick_params(axis="x", rotation=45)
pivot_gates.plot(kind="bar", ax=axs[1], legend=False)
axs[1].set_title("2Q Gate Count Comparison")
axs[1].set_ylabel("Number of 2Q Gates")
axs[1].set_xlabel("Pattern")
axs[1].tick_params(axis="x", rotation=45)
pivot_time.plot(
    kind="bar", ax=axs[2], legend=True, title="Legend"
)  # Show legend on the last plot
axs[2].set_title("Time Comparison")
axs[2].set_ylabel("Time (seconds)")
axs[2].set_xlabel("Pattern")
axs[2].tick_params(axis="x", rotation=45)
fig.suptitle(
    "Benchmarking AI Synthesis Methods vs Non-AI Synthesis Methods For Random Permutations Circuits",
    fontsize=16,
    y=1,
)

plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/a326f268-0115-442c-8563-968676b66670-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/ai-transpiler-introduction/extracted-outputs/79b8d5d9-0f9d-42ca-9583-8bec17430014-0.avif)

AIトランスパイラの性能は、最適化される回路の種類によって大きく異なります。一部のケースでは、標準トランスパイラと比較して回路深度とゲート数の顕著な削減を達成します。しかし、これらの改善には多くの場合、実行時間の大幅な増加が伴います。

特定の種類の回路では、AIトランスパイラは回路深度においてわずかに良い結果を得る場合がありますが、ゲート数の増加や実行時間の大幅なペナルティをもたらす可能性もあります。これらの観察結果は、AIトランスパイラの利点がすべての回路タイプに一律に適用されるわけではないことを示唆しています。その有効性は回路の具体的な特性に依存するため、特定のユースケースには他よりも適しています。

## ユーザーはいつAI搭載トランスパイレーションを選択すべきですか？

QiskitのAI搭載トランスパイラは、従来のトランスパイレーション手法では困難なシナリオ、特に大規模で複雑な量子回路において優れた性能を発揮します。数百量子ビットを含む回路や、複雑な結合マップを持つハードウェアをターゲットとする回路に対して、AIトランスパイラは回路深度、ゲート数、および実行時間効率の面で優れた最適化を提供します。ベンチマークテストでは、従来の手法を一貫して上回り、大幅に浅い回路を生成し、ゲート数を削減しています。これらは、実際の量子ハードウェア上でのパフォーマンス向上とノイズ軽減にとって極めて重要です。

ユーザーは以下のような場合にAI搭載トランスパイレーションを検討すべきです：
- 従来の手法ではスケールを効率的に処理できない大規模回路。
- デバイスの接続性やルーティングの課題が生じる複雑なハードウェアトポロジー。
- 回路深度の削減と忠実度の向上が最優先されるパフォーマンスが重要なアプリケーション。

# パートIII. AI搭載順列ネットワーク合成の探索

順列ネットワークは量子コンピューティングの基盤であり、特に制限されたトポロジーに制約されるシステムにおいて重要です。これらのネットワークは、限られた接続性を持つハードウェア上で全対全の接続性を模倣するために、量子ビットを動的にスワップすることで長距離相互作用を実現します。このような変換は、相互作用が最近傍を超えることが多い近接項デバイスにおいて、複雑な量子アルゴリズムを実装するために不可欠です。

このセクションでは、QiskitのAI搭載トランスパイラの魅力的なユースケースとして、順列ネットワークの合成を紹介します。具体的には、`AIPermutationSynthesis`パスがAI駆動の最適化を活用して、量子ビット順列タスクのための効率的な回路を生成します。一方、汎用的な合成アプローチでは、特に密な量子ビット相互作用を伴うシナリオや完全な接続性の実現を試みる場合に、ゲート数と回路深度のバランスを取ることが困難になりがちです。

ここでは、量子ビットセットの全対全接続性を実現するための順列ネットワークの合成を示すQiskitパターンの例を説明します。`AIPermutationSynthesis`とQiskitの標準合成手法のパフォーマンスを比較します。この例では、AIトランスパイラがより低い回路深度とゲート数に対してどのように最適化するかを示し、実用的な量子ワークフローにおけるその利点を強調します。AI合成パスを有効にするには、`generate_ai_pass_manager()`関数の`include_ai_synthesis`パラメータを`True`に設定して使用します。

## ステップ1：古典的な入力を量子問題にマッピングする

古典的な順列問題を量子コンピュータ上で表現するために、まず量子回路の構造を定義します。この例では：

1. 量子回路の初期化：
   使用するバックエンドの27量子ビットに合わせて、27量子ビットを割り当てます。

2. 順列の適用：
   再現性のために固定シードを使用して、10個のランダムな順列パターン（`pattern_1`から`pattern_10`）を生成します。各順列パターンは、個別の量子回路（`qc_1`から`qc_10`）に適用されます。

3. 回路の分解：
   各順列操作は、ターゲット量子ハードウェアと互換性のあるネイティブゲートセットに分解されます。分解された各回路の深度と2量子ビットゲート数（非局所ゲート）を分析します。

これらの結果は、量子デバイス上で古典的な順列問題を表現する際の複雑さに関する知見を提供し、異なる順列パターンに対するリソース要件を示します。